Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
IMMATRICULATION_NUMBER = ""

---

## Exercise 9: Graph compression with k<sup>2</sup> trees

Goals of this exercise: 
- get a deep understanding of the representation of binary matrices by k<sup>2</sup> trees 
- get a deep understanding of k<sup>2</sup> tree storage in levels 


## 1. Preliminaries		

Implement a method fill that can be called by 
```
fill(Matrix)
```
that fills a Matrix `Matrix` of size `N x N` with rows and columns of 0 such that `Matrix` is enlarged size  `newSize x newSize`, with `newSize` is the smallest power of 2 that is bigger than or equal to `N`. Furthermore, the method should return the new size `newSize`. You can assume that the Matrix `Matrix` has a minimum size of `2 x 2`

For example 
```
M = 
[
[1, 0],
[0, 1]
]
```
remains unchanged, 

```
M = 
[
[1, 1, 0],
[0, 0, 0],
[0, 0, 1]
]
```
is filled to

```
M = 
[
[1, 1, 0, 0],
[0, 0, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 0]
]
```

In [2]:
import math

def fill(Matrix):
    # assert is square matrix
    assert len(Matrix) == len(Matrix[0])
    n = len(Matrix)
    next_2_power = 2**math.ceil(math.log(n, 2))
    for i in range(n):
        Matrix[i].extend([0]*(next_2_power-n))
    for i in range(n, next_2_power):
        Matrix.append([0]*next_2_power)
    return next_2_power


In [3]:
import copy

M1_1 = [
[1, 0],
[0, 1]
]
M1_2 = copy.deepcopy(M1_1)
assert fill(M1_1)==2
assert M1_2==M1_1

M2_1 = [
[1, 1, 0],
[0, 0, 0],
[0, 0, 1]
]
M2_2 = [
[1, 1, 0, 0],
[0, 0, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 0]
]
assert fill(M2_1)==4
assert M2_2==M2_1

M3_1 = [
[1, 0, 0, 0, 0],
[0, 1, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 1, 1, 1],
[0, 1, 0, 0, 0]
]
M3_2 = [
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 1, 1, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0]
]
assert fill(M3_1)==8
assert M3_2==M3_1

## 2. Compute the k<sup>2</sup> tree.  

Implement a method build that can be called by 
```
	k2treeNode = build(Levels,level,Matrix,row,column,N) 
```
and that computes the bits of all Levels of a k<sup>2</sup> tree for a given input `Matrix`. 
The method build shall recursively evaluate the bits in that `N x N` submatrix of `Matrix` which has its top-left corner at the indices `[row,column]`. If this `N x N` submatrix contains only values 0, the return value of `k2treeNode` shall become 0, otherwise the return value of `k2treeNode` shall become 1. 

Furthermore, whenever the return value for `k2treeNode` is 1 and that node is not a leaf node of the k<sup>2</sup> tree, the Levels array shall be extended as a side effect by the 4 bits representing the 4 children of this node in the k<sup>2</sup> tree. 
The computed `k2treeNode` values shall NOT be stored in a tree, but shall be directly stored in the correct level of the `Levels` array which is passed as the first parameter.  `Levels` shall be organized as a list of bit lists, where each bit list represents the bits of one level of the k<sup>2</sup> tree. 

For example, if 
```
Matrix = [
[0, 0, 0, 1],
[1, 0, 1, 1],
[0, 0, 1, 0],
[0, 0, 1, 0]
]
```
and `Levels` is initially empty, after a call of 
```
	k2treeNode = build(Levels,0,Matrix,0,0,4)
```
`Levels` should contain the following lists of numbers:`
```
	Levels[0] = [1,1,0,1] 
	Levels[1] = [0,0,1,0,0,1,1,1,1,0,1,0] 
```

You shall assume that the input matrix always has a width and height that is a power of 2.


In [6]:
def build(Levels,level,Matrix,row,col,N): # build Levels of k2 tree from Matrix
    if N <= 1:
        return Matrix[row][col]
    if level > len(Levels ):
        raise Exception("Level out of range")
    # submatrix = Matrix[row:row+N]
    # submatrix = [x[col:col+N] for x in submatrix]
    ret = 0
    results = []
    if len(Levels) == level:
        Levels.append([])
    for x in range(2):
        for y in range(2):
            # sub_submatrix = submatrix[x*N//2:(x+1)*N//2,y*N//2:(y+1)*N//2]
            # sub_submatrix = [submatrix[i][y*N//2:(y+1)*N//2] for i in range(x*N//2,(x+1)*N//2)]
            # using build, check if sub_submatrix is all 0
            res = build(Levels,level+1,Matrix,row+x*N//2,col+y*N//2,N//2)
            results.append(res)
            if res:
                ret = 1
    # print(f"Level {Levels=} {level=}, {row=}, {col=}, {N=}, {results=} {ret=}")
    if ret:
        Levels[level] += results
    return ret


In [7]:
Matrix = [
[0, 0, 0, 1],
[1, 0, 1, 1],
[0, 0, 1, 0],
[0, 0, 1, 0]
]
Levels=[[1,1,0,1],[0,0,1,0,0,1,1,1,1,0,1,0]]
L = []
build(L, 0, Matrix, 0, 0, 4)
assert L==Levels, f"Test failed for input {Matrix=}, {L=}, {Levels=}"

import random

def matrix(n):                 # return a matrix filled with random numbers 
    return [[int(1/0.80 * random.random()) for _ in range(n)] for _ in range(n)]

def pp(Matrix):              # pretty print Matrix
    print("[")
    for i in range(len(Matrix)):
        print("%s," % Matrix[i])
    print("]")

        
def retransform(Levels):
    M = Levels[0]
    for nextLevel in range(1, len(Levels)):
        _M = []
        for e in M:
            if e==1:
                _M.extend(Levels[nextLevel][:4])
                del Levels[nextLevel][:4]
            else: _M.extend([0,0,0,0])
        M = _M
    return M

def retransform2(M, r, c, s): 
    if(s==1): return [M[r][c]]
    else:
        flat = []
        for i in range(2):
            for j in range(2):
                n=s//2
                flat.extend(retransform2(M,r+i*n,c+j*n,n))
        return flat
        
def test(size, rep):
    for s in range(2,size):
        for r in range(rep):
            M = matrix(s)
            while(sum([sum(i) for i in M])==0): M = matrix(s)
            
            newSize = fill(M)

            Levels = []
            build(Levels, 0, M, 0, 0, newSize)
            newM = retransform(Levels)
            
            newM2 = retransform2(M,0,0,newSize)
            assert newM2==newM, "Test failed for input %s" % M
    print("All tests were performed successfully!")  

test(9, 3)

All tests were performed successfully!
